# Excitation Generation via ForecastPFN

在该示例中我们展示了如果基于[`ForecastPFN`](https://arxiv.org/abs/2311.01933)中提出的时间序列组合的方式来生成激励信号。该论文主要基于先验特征拟合网络的方式在大量合成数据中训练了一个零样本时间序列预测模型。

在S2Generator中我们对原本代码中的数据生成方法进行了改进，并提供了Python形式的面向对象编程接口使其更加方便进行参数管理和调用。其中原始的代码链接为: https://github.com/abacusai/ForecastPFN.

该方法认为时间序列$y_t$是由趋势和大量不同频率的周期分量组成的。这些周期分量包括了星期、月份和年份。并最后向生成的序列数据添加部分随机的噪声。对于目标生成的时间序列$y_t$我们可以将其看作趋势和周期的组合:

$$
y_t = \phi(t) \cdot z_t = \mathrm{trend}(t) \cdot \mathrm{seasonal}(t) \cdot z_t,
$$
where, $z_t$ is the noise time series and can be represented as

$$
z_t = 1 + m_{\mathrm{noise}}(z - \bar{z}),
$$
where, $z \sim \mathrm{Weibull}(1, k)$, $\bar{z} = (\mathrm{ln}2) ^ {1 / k}$.

The trend component is made up of a linear and exponential component with coefficients:
$$
\mathrm{trend}(t) = (1 + m_{\mathrm{linear}} \cdot t + c_{\mathrm{linear}})(m_{\mathrm{exp}} \cdot c_{\mathrm{exp} ^ t}),
$$
where, $m_{\mathrm{linear}}, m_{\mathrm{exp}} ~ \mathcal{N}(-0.01, 0.5)$, $c_{\mathrm{linear}} \sim \mathcal{N}(0, 0.01)$, $c_{\mathrm{exp}} \sim \mathcal N (1, 0.0005)$.

